# Diebold & Mariano (1995) Heatmaps

The DM_Test_Results function takes in three arguments. First, prediction_path is a .csv file path that contains the predicted values from all forecasting algorithms. Second, result_path is a .csv file path that contains the ranked RMSE values for each forecasting algorithm. Lastly, actual_series_name represents the target variable to be forecasted. The output of the DM_Test_Results function produces a heatmap that contains the Diebold & Mariano (1995) p-values for equal predictive accuracy for a proposed model relative to a baseline model. Each model is allowed to act as the baseline model, which means all pairwise comparisons are considered.   

In [ ]:
# Load Library:
from pandas import read_csv
import pandas as pd
import numpy as np
from dieboldmariano import dm_test
from matplotlib import pyplot
import seaborn as sns
# Define Function to Compute & Store Diebold-Mariano Test Results:
def DM_Test_Results(prediction_path, result_path, actual_series_name):
    # Load In the Prediction Data From Path:
    prediction_data = read_csv(prediction_path, header = 0, index_col = 0, parse_dates = True)
    prediction_data.index = pd.DatetimeIndex(prediction_data.index.values, freq = "MS")
    # Load in Model Results Data:
    result_data = read_csv(result_path, header = 0, index_col = 0, usecols = ['Model_Name', 'RMSE'])
    proposed_models = result_data.index
    baseline_models = proposed_models
    # Create Storage Bin for DM Test Results:
    DM_Results = []
    for i in range(len(baseline_models)):
        for j in range(len(proposed_models)):
            if proposed_models[j] == baseline_models[i]:
                DM_Results.append(np.nan)
            else:
                test_results = dm_test(prediction_data[actual_series_name], prediction_data[proposed_models[j]], prediction_data[baseline_models[i]], harvey_correction = False, one_sided = True)
                DM_Results.append(round(test_results[1],3))
    DM_Results = np.asarray(DM_Results).reshape((len(baseline_models),len(proposed_models)))
    DM_Results = pd.DataFrame(DM_Results, index = baseline_models, columns = proposed_models)
    # Define the Heatmap:
    pyplot.figure(figsize = (16,12))
    sns.heatmap(DM_Results, mask = DM_Results.isnull(), linewidths=.5, cbar_kws={'orientation':'vertical', 'pad': 0.005})
    pyplot.xticks(fontsize = 12)
    pyplot.yticks(fontsize = 12)
    pyplot.xlabel('Proposed Model', fontsize = 14)
    pyplot.ylabel('Baseline Model', fontsize = 14)
    pyplot.title('Pairwise Diebold & Mariano (1995) Forecast Comparison Test', fontsize = 18)
    pyplot.tight_layout()
    pyplot.show()
# Set Prediction & Result Paths:
Prediction_Path = 'National_Test_Growth_One.csv'
Result_Path = 'National_Test_Growth_One_RMSE.csv'
# Set Target Series:
Actual_Series = 'RHP'
# Compute, Store, & Return Diebold-Mariano Test Heatmap:
DM_Test_Results(Prediction_Path, Result_Path, Actual_Series)